In [3]:
pip install -r requirements.txt

  Using cached asyncio-4.0.0-py3-none-any.whl.metadata (994 bytes)
  Using cached openai-2.20.0-py3-none-any.whl.metadata (29 kB)
  Using cached scipy-1.17.0-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached transformers-5.1.0-py3-none-any.whl.metadata (31 kB)
  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached sentence_transformers-5.2.2-py3-none-any.whl.metadata (16 kB)
  Using cached numpy-2.1.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached bert_score-0.3.13-py3-none-any.whl.metadata (15 kB)
  Using cached scikit_learn-1.8.0-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (11 kB)
  Using cached rouge-1.0.1-py3-none-any.whl.metadata (4.1 kB)
  Using cached razdel-0.5.0-py3-none-any.whl.metadata (10.0 kB)
  Using cached jiter-0.

In [8]:
lst = list(await bench.client.client.models.list())[0][1]
for l in lst:
    print(l)

Model(id='ruadapt-qwen3-4b', created=None, object=None, owned_by=None, max_model_len=40960, status='spawned')
Model(id='RefalMachine/RuadaptQwen3-32B-Instruct-v2', created=None, object=None, owned_by=None, max_model_len=24000, status='spawned')
Model(id='qwen3moe30b', created=None, object=None, owned_by=None, max_model_len=20000, status='spawned')
Model(id='tpro-2', created=None, object=None, owned_by=None, max_model_len=12000, status='spawned')
Model(id='DeepSeek V3', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='Qwen3-235B-A22B-Instruct-2507', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='qwen3next-instruct', created=None, object=None, owned_by=None, max_model_len=60000, status='offloaded')
Model(id='gemma3-4b', created=None, object=None, owned_by=None, max_model_len=40960, status='offloaded')
Model(id='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', created=None, object=None, owned_by=None, max_model_len=

In [1]:
from utils import chunk_text
import sys
import torch
from sentence_transformers import SentenceTransformer
sys.path.append('src')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()[:2]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)
bench = Summarisation(url=url.strip(), key=key.strip(), model_name='ruadapt-qwen3-4b', device=device, encoder=encoder)
bench.prepare_environment()

result = await bench.run_benchmark_one_method(
    is_evalutation_needed=True,
    number_of_books=1,
    method='hierarchical',
    mode='filtered',
    initial_word_limit=500,
    cap_chars=80000,
    output_dir='output'
)

C:\Users\dagri\anaconda3\envs\BoookSum\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dagri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\dagri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Loading weights: 100%|██████████| 391/391 [00:00<00:00, 690.42it/s, Materializing param=pooler.dense.weight]                               


   book_idx       book_title book_author       book_genre        method  \
0         0  Башня Волверден       Аллен  Science Fiction  hierarchical   

       mode  initial_word_limit  text_len (words)  annotation_len (words)  \
0  filtered                 250              8849                     208   

   runtime_sec  bertscore_p  bertscore_r  bertscore_f    rougeL  
0      20.3378     0.683016     0.562777     0.617094  0.103614  


In [1]:
import sys
import torch
from sentence_transformers import SentenceTransformer
sys.path.append('src')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()[:2]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)

/home/grigorievda/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /home/grigorievda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/grigorievda/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Loading weights: 100%|█| 391/391 [00:00<00:00, 846.04it/s, Materializing param=p


In [2]:
from utils import AsyncList

async def run_bench_for_model(
    url: str,
    key: str,
    model_name: str,
    device = None,
    encoder = None,
    output_dir: str = 'output',
    errors_dir: str = 'errors',
    is_evaluation_needed: bool = True,
    number_of_books: int = 25,
    cap_chars: int = 80000,
    method: str = 'hierarchical',
    mode: str = 'default',
    initial_word_limit: int = 500
):
    bench = Summarisation(url=url, key=key, model_name=model_name, device=device, encoder=encoder)
    bench.prepare_environment()
 
    result = await bench.run_benchmark_one_method(
        is_evaluation_needed=is_evaluation_needed,
        number_of_books=number_of_books,
        method=method,
        mode=mode,
        initial_word_limit=initial_word_limit,
        cap_chars=cap_chars,
        output_dir=output_dir,
        errors_dir=errors_dir
    )


async def run_all_models(
    url: str,
    key: str,
    device,
    encoder,
    is_evaluation_needed: bool = True,
    number_of_books: int = 25,
    cap_chars: int = 80000,
    method: str = 'hierarchical',
    mode: str = 'default',
    initial_word_limit: int = 500,
    main_output_dir: str = 'output',
    errors_dir: str = 'errors'
):
    models = AsyncList()
    model_names = [
        'ruadapt-qwen3-4b',
        'RefalMachine/RuadaptQwen3-32B-Instruct-v2',
        'Qwen3-235B-A22B-Instruct-2507',
        'DeepSeek V3',
        'tpro-2'
    ]
    for model_name in model_names:
        output_dir = f'{main_output_dir}/{model_name.replace('/', '_')}_results'
        models.append(run_bench_for_model(
            url=url,
            key=key,
            model_name=model_name,
            device=device,
            encoder=encoder,
            output_dir=output_dir,
            errors_dir=errors_dir,
            is_evaluation_needed=is_evaluation_needed,
            number_of_books=number_of_books,
            cap_chars=cap_chars,
            method=method,
            mode=mode,
            initial_word_limit=initial_word_limit
        ))

    await models.complete_couroutines(batch_size=len(model_names))
    models = await models.to_list()

In [3]:
await run_all_models(
    url=url,
    key=key,
    device=device,
    encoder=encoder,
    is_evaluation_needed=True,
    number_of_books=700,
    cap_chars=-1,
    method='blueprint',
    mode='default',
    initial_word_limit=500,
    main_output_dir='blueprint_default',
    errors_dir='errors_new'
)

CancelledError: 